In [310]:
import pandas as pd
import numpy as np

masterImports = pd.read_excel('Data/DETALLE SAINT GOBAIN_copia.xlsx', sheet_name='ENTRADAS 2024')

In [311]:
# Prueba de Header
masterImports.fillna(0, inplace=True)
masterImports.head(2)


,#,Fecha de descarga,BL No.,AID,Origen,REFERENCIA,Puerto de llegada,Contenedor,Tamaño,DMC $125.00,...,TI,Gastos Portuarios,Otros /Flete,Documentacion de aduana,Valor F.O.B.,ASN OK?,excedente,Faltante,Abolladura,Producto afectado
0,1,2024-01-04,SSZ1410999 / SGB202311002,118,BRASIL,SSZ1410999 / SGB202311002,MANZANILLO,TEXU9539902,40,20230702186,...,0.0,47,0.0,25 TI,100599.95,0,0,0,0,0
1,2,2024-01-17,MIA/PTY/E23220,119,ESTADOS UNIDOS,MIA/PTY/E23220,MANZANILLO,724816,26,20230698394,...,0.0,0,15.0,0,15408.92,0,0,0,0,0


# Apartado de Imports

Acumulado

In [312]:
totalCajas = int(masterImports['CAJAS'].sum())
totalCBM = float(masterImports['CBM'].sum())
totalPaletas = int(masterImports['Total Paletas'].sum())
palSKU = int(masterImports[ 'PALETAS 1 SKU'].sum())


print('Total de cajas: ', totalCajas)	
print('Total de CBM: ',  totalCBM)
print('Total de Paletas: ', totalPaletas)


Total de cajas:  111765
Total de CBM:  721.5899999999999
Total de Paletas:  728


Cuadro de Embarques

In [313]:
fechaDescarga = masterImports['Fecha de descarga']
fechaDescarga = fechaDescarga.dt.strftime('%B')  # Formatear la fecha para obtener solo el mes y el año

# Crear la tabla pivote con los meses como columnas
contRecibidos = pd.pivot_table(masterImports, index=fechaDescarga, columns='Tamaño', values='#', aggfunc='count').fillna(0)
contRecibidos = contRecibidos.transpose()

contRecibidos


Fecha de descarga,April,August,February,January,July,March,May
Tamaño,,,,,,,
20,0.0,1.0,2.0,0.0,1.0,0.0,0.0
26,3.0,0.0,0.0,2.0,0.0,0.0,0.0
40,2.0,2.0,0.0,2.0,4.0,4.0,5.0
LCL,0.0,1.0,0.0,0.0,3.0,0.0,0.0


In [314]:
# Sumar los valores de cada columna (mes) en contRecibidos
# Traducir los nombres de los meses al español
traduccion_meses = {
    'January': 'Enero', 'February': 'Febrero', 'March': 'Marzo', 'April': 'Abril',
    'May': 'Mayo', 'June': 'Junio', 'July': 'Julio', 'August': 'Agosto',
    'September': 'Septiembre', 'October': 'Octubre', 'November': 'Noviembre', 'December': 'Diciembre'
}

# Sumar los valores de cada columna (mes) en contRecibidos
suma_meses = contRecibidos.sum()

# Convertir el resultado en una serie
suma_meses = pd.Series(suma_meses)

# Ordenar la serie por el orden de los meses
orden_meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
suma_meses = suma_meses.reindex(orden_meses).dropna()

# Traducir los nombres de los meses al español
suma_meses.index = suma_meses.index.map(traduccion_meses)

print(suma_meses)



Fecha de descarga
Enero      4.0
Febrero    2.0
Marzo      4.0
Abril      5.0
Mayo       5.0
Julio      8.0
Agosto     4.0
dtype: float64


In [315]:
sumEnero = suma_meses['Enero']
sumFebrero = suma_meses['Febrero']
sumMarzo = suma_meses['Marzo']
sumAbril = suma_meses['Abril']
sumMayo = suma_meses['Mayo']
sumJulio = suma_meses['Julio']
sumAgosto = suma_meses['Agosto']

print(f"Enero: {sumEnero}, Febrero: {sumFebrero}, Marzo: {sumMarzo}, Abril: {sumAbril}, Mayo: {sumMayo}, Julio: {sumJulio}, Agosto: {sumAgosto}")

Enero: 4.0, Febrero: 2.0, Marzo: 4.0, Abril: 5.0, Mayo: 5.0, Julio: 8.0, Agosto: 4.0


In [326]:
import json

# Convertir el DataFrame a un diccionario
contRecibidos_dict = contRecibidos.to_dict()

# Convertir el diccionario a un JSON
contRecibidos_json = json.dumps(contRecibidos_dict, indent=4)

# Guardar el JSON en un archivo
with open('data/json/contRecibidos.json', 'w') as json_file:
    json_file.write(contRecibidos_json)

print("El DataFrame se ha convertido y guardado en contRecibidos.json")

El DataFrame se ha convertido y guardado en contRecibidos.json


Contenedores recibidos por país

In [317]:
contMeses = pd.pivot_table(masterImports, index=fechaDescarga, columns='Origen', values='#', aggfunc='count').fillna(0)
contMeses = contMeses.transpose()

contMeses

Fecha de descarga,April,August,February,January,July,March,May
Origen,,,,,,,
ALEMANIA,0.0,0.0,0.0,0.0,1.0,0.0,0.0
ARGENTINA,0.0,1.0,0.0,0.0,0.0,0.0,1.0
ASIA,0.0,0.0,0.0,0.0,1.0,0.0,0.0
BELGICA,1.0,0.0,0.0,0.0,0.0,0.0,0.0
BRASIL,1.0,0.0,1.0,1.0,3.0,3.0,2.0
CHINA,1.0,1.0,0.0,1.0,0.0,0.0,0.0
COLOMBIA,1.0,2.0,1.0,1.0,1.0,1.0,2.0
ESTADOS UNIDOS,1.0,0.0,0.0,1.0,1.0,0.0,0.0
HOLANDA,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [322]:
origenes = masterImports['Origen']
origenes = origenes.value_counts()

origenes

import json

origenes_dict = origenes.to_dict()
origenes_json = json.dumps(origenes_dict, indent=4)

with open('data/json/origenes.json', 'w') as json_file:
    json_file.write(origenes_json)
    
    
print("El archívo 'origenes.json' ha sido creado con éxito")

El archívo 'origenes.json' ha sido creado con éxito


Cajas, CBM, KG y Paletas mensuales recibidas

In [319]:
kg = masterImports['PESO']
kgMensual = pd.pivot_table(masterImports, index=fechaDescarga, values='PESO', aggfunc='sum').fillna(0)
kgMensual = kgMensual.transpose()

kg_total = kg.sum()

print("El peso total es: ",kg_total, "| La distribución mensual es: ")
kgMensual

El peso total es:  428516.05 | La distribución mensual es: 


Fecha de descarga,April,August,February,January,July,March,May
PESO,34715.36,59280.89,24961.32,44609.72,97857.23,87542.8,79548.73


In [320]:
cbm = masterImports['CBM']
cbmMensual = pd.pivot_table(masterImports, index=fechaDescarga, values='CBM', aggfunc='sum').fillna(0)
cbmMensual = cbmMensual.transpose()

cbm_total = cbm.sum()

print("El CBM total es: ",cbm_total, "| La distribución mensual es: ")
cbmMensual

El CBM total es:  721.5899999999999 | La distribución mensual es: 


Fecha de descarga,April,August,February,January,July,March,May
CBM,62.158,136.064,37.852,101.56,156.486,148.696,78.774


In [321]:
paletasSku = masterImports['PALETAS 1 SKU']
paletasMixed = masterImports['PALETAS MIX SKU']

# totalPaletas = paletasSku.sum() + paletasMixed.sum()
masterImports['totalPaletas'] = masterImports['PALETAS 1 SKU'] + masterImports['PALETAS MIX SKU']

totalPaletasMes = pd.pivot_table(masterImports, index=fechaDescarga, values='totalPaletas', aggfunc='sum').fillna(0)
totalPaletasMes = totalPaletasMes.transpose()

totalPaletasMes
# print(masterImports.head(2))

Fecha de descarga,April,August,February,January,July,March,May
totalPaletas,65.0,73.0,36.0,138.0,161.0,140.0,127.0
